# Inception v3

In [ ]:
!pip install timm

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

from torchvision import models
import torchvision.utils
import torchvision.datasets as dsets
import torchvision.transforms as transforms

from timm.loss import LabelSmoothingCrossEntropy

import numpy as np
import time
import copy
import sys
from tqdm import tqdm

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

## Prepare Data

In [ ]:
# Data augementation pipline
transform = transforms.Compose([transforms.RandomHorizontalFlip(0.9),
    transforms.Resize((299, 299)),
    transforms.ToTensor(), # ToTensor : [0, 255] -> [0, 1]
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])
​
# Train transformation pipeline
train_transform = transforms.Compose([
    transforms.Resize((299, 299)),
    transforms.ToTensor(), # ToTensor : [0, 255] -> [0, 1]
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])
​
# Test transformation pipeline
test_transform = transforms.Compose([
    transforms.Resize((299, 299)),
    transforms.ToTensor(), # ToTensor : [0, 255] -> [0, 1]
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])
train_data1 = dsets.ImageFolder('/content/drive/MyDrive/train/', transform)
test_data1 = dsets.ImageFolder('/content/drive/MyDrive/val/',  transform)
train_data2 = dsets.ImageFolder('/content/drive/MyDrive/train/', train_transform)
test_data2 = dsets.ImageFolder('/content/drive/MyDrive/val/', test_transform)

train_data = torch.utils.data.ConcatDataset([train_data1,train_data2])
test_data = torch.utils.data.ConcatDataset([test_data1,test_data2])

In [ ]:
batch_size = 11

train_loader = DataLoader(train_data,
                          batch_size=batch_size,
                          shuffle=True, drop_last=True)

test_loader = DataLoader(test_data,
                         batch_size=batch_size,
                         shuffle=True, drop_last=True)

In [ ]:
train_data_len = len(train_data)
test_data_len = len(test_data)

dataloaders = {
    "train": train_loader,
    "val": test_loader
}
dataset_sizes = {
    "train": train_data_len,
    "val": test_data_len
}

## Define Model

In [ ]:
model = models.inception_v3(pretrained=True)

In [ ]:
model

In [ ]:
# Set parameters trainable/non-trainable
model.aux_logits = False

for parameter in model.parameters():
    parameter.requires_grad = False

In [ ]:
# To be used when fine-tuning on Batch Norm
for parameter in model.parameters():
    parameter.requires_grad = True
    if isinstance(parameter, nn.BatchNorm2d):
        parameter.requires_grad = False

In [ ]:
# Alternatively replace batch norm with layer norm
def convert_layers(model, layer_type_old, layer_type_new, convert_weights=True, num_groups=None):
    for name, module in reversed(model._modules.items()):
        if len(list(module.children())) > 0:
            # recurse
            model._modules[name] = convert_layers(module, layer_type_old, layer_type_new, convert_weights)

        if type(module) == layer_type_old:
            layer_old = module
            layer_new = layer_type_new(module.num_features if num_groups is None else num_groups, module.num_features, module.eps, module.affine)

            if convert_weights:
                layer_new.weight = layer_old.weight
                layer_new.bias = layer_old.bias

            model._modules[name] = layer_new

    return model

model = convert_layers(model, torch.nn.BatchNorm2d, torch.nn.GroupNorm, num_groups = 4)



In [ ]:
# When fine-tuning with Group Norm
for parameter in model.parameters():
    parameter.requires_grad = True
    if isinstance(parameter, nn.GroupNorm):
        parameter.requires_grad = False


In [ ]:
# Fully connected classifier head
model.fc = nn.Sequential(
    nn.Linear(model.fc.in_features, 512),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(512, 4)
)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device
model = model.to(device)

In [ ]:
loss_c = LabelSmoothingCrossEntropy()
optimizer = torch.optim.RMSprop(filter(lambda p: p.requires_grad, model.parameters()), lr=0.0045)
num_epochs = 30
#exp_lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.97)
exp_lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, 100)

## Train Model

In [ ]:
def train_model(model, loss_c, optimizer, scheduler, num_epochs=50):
    since = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print("-"*10)

        for phase in ['train', 'val']: # We do training and validation phase per epoch
            if phase == 'train':
                model.train() # model to training mode
            else:
                model.eval() # model to evaluate

            running_loss = 0.0
            running_corrects = 0.0

            for inputs, labels in tqdm(dataloaders[phase]):
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'): # no autograd makes validation go faster
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1) # used for accuracy
                    loss = loss_c(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            if phase == 'train':
                scheduler.step() # step at end of epoch

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc =  running_corrects.double() / dataset_sizes[phase]

            print("{} Loss: {:.4f} Acc: {:.4f}".format(phase, epoch_loss, epoch_acc))

            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict()) # keep the best validation accuracy model
        print()
    time_elapsed = time.time() - since # slight error
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print("Best Val Acc: {:.4f}".format(best_acc))

    model.load_state_dict(best_model_wts)
    return model

In [ ]:
model_ft = train_model(model, loss_c, optimizer, exp_lr_scheduler)

## Test Model

In [ ]:
model.eval()

correct = 0
total = 0

for images, labels in test_loader:

    images = images.cuda()
    outputs = model(images)

    _, predicted = torch.max(outputs.data, 1)

    total += labels.size(0)
    correct += (predicted == labels.cuda()).sum()

print('Accuracy of test images: %f %%' % (100 * float(correct) / total))

# Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sn
import pandas as pd

y_pred = []
y_true = []

# iterate over test data
for images, labels in train_loader:

        images = images.cuda()
        output = model(images) # Feed Network

        output = (torch.max(torch.exp(output), 1)[1]).data.cpu().numpy()
        y_pred.extend(output) # Save Prediction

        labels = labels.data.cpu().numpy()
        y_true.extend(labels) # Save Truth

# constant for classes
classes = ("High_risk", "Low_risk","Medium_risk","invalid")

# Build confusion matrix
cf_matrix = confusion_matrix(y_true, y_pred)
df_cm = pd.DataFrame(cf_matrix / np.sum(cf_matrix, axis=1)[:, None], index = [i for i in classes],
                     columns = [i for i in classes])
plt.figure(figsize = (12,7))
sn.heatmap(df_cm, annot=True)
plt.savefig('output.png')

In [ ]:
classes = ["High_risk", "Medium_risk","Low_risk", "invalid"]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
images, labels = iter(test_loader).next()

outputs = model(images.cuda())

_, predicted = torch.max(outputs.data, 1)

print('Predicted: ', ' '.join('%5s' % classes[predicted[j]] for j in range(5)))

title = (' '.join('%5s' % classes[labels[j]] for j in range(5)))
imshow(torchvision.utils.make_grid(images, normalize=True), title)